# 📊 Análisis de Casos COVID ‑ Formosa: Flujo ETL y Exploración de Datos

> ⚙️ *Este notebook concentra el ciclo completo de procesamiento masivo de datos COVID‑19 en Argentina, con foco en la provincia de Formosa.*  
> 💾 **Volumen inicial:** 6 GB, 30 millones de registros  
> 🗓️ **Período:** 01/03/2020 – 04/06/2022

---

## 📋 Resumen General del Dataset

- 🔢 **Total de registros:** `797.429`
- 📄 **Total de columnas:** `25`  
- 📌 **Alcance:** Dataset filtrado exclusivamente para la provincia de Formosa
- 🧠 **Objetivo:** Comprender la evolución temporal, distribución geográfica y características demográficas de los casos COVID en Formosa

---

## 🧭 1. ¿Por qué este análisis?

Este trabajo consiste en seleccionar un dataset de preferencia, en este caso vamos a trabajar con el dataset que cuenta con la información de "Casos de Covid19 durante la pandemia"

limitando el alcance del dataset y trabajando unicamente con los datos que pertenezcan a la provincia de Formosa.


---

## 🎯 2. Objetivos

### 🔄 Flujo ETL Completo

1. **📥 Extracción**: Leer los datos por bloques y seleccionar solo Formosa.  
2. **🧹 Transformación**: Estandarizar, limpiar y validar.  
3. **💾 Carga**: Exportar los datos listos para análisis.

### 🔬 Exploración y Visualización

4. **📊 Estadísticas**: 
   - Casos totales por departamento y localidad
   - Fallecimientos por grupo etario y género
   - Clasificaciones de casos (confirmados, sospechosos)
   - Pacientes en cuidados intensivos
   - Tiempo entre diagnóstico y primeros síntomas
   - Casos por semana epidemiológica
   - Índice de mortalidad en personas jóvenes (<30 años)
   - Uso de asistencia respiratoria mecánica por edad

5. **📈 Gráficos**: 
   - Series de tiempo para evolución de casos
   - Distribución geográfica de severidad
   - Análisis demográfico por género y edad
   - Comparativas entre clasificaciones y resultados

---

## 🛠️ 3. Metodología ETL Unificada

### 📤 3.1 Ingestión por Fragmentos

- ✅ Selección de columnas clave: `provincia`, `fecha_inicio_sintomas`, `clasificacion`, `fallecido`.  
- ✅ Filtro temprano: conservar solo **Formosa**.

### 🧼 3.2 Limpieza de Datos

- 🗃️ Normalización de fechas y categorías.  

## 📊 4. Análisis Exploratorio

### 📌 4.1 Estadísticas Básicas

- 🔢 Total de casos registrados por departamento.  
- ⚰️ Cantidad de fallecimientos por grupo etario y género.  
- 🧾 Clasificación por tipo de caso (confirmado, sospechoso, etc.).
- 🏥 Casos que requirieron atención en UCI.

### 📅 4.2 Tendencias Temporales

- 📆 Evolución diaria y acumulada.  
- 🚨 Detección de picos y brotes.
- 📊 Cantidad de infectados por semana epidemiológica.

### 🧮 4.3 Métricas Derivadas

- ⏱️ Intervalo entre diagnóstico y aparición de síntomas.
- 👶 Índice de mortalidad en personas menores de 30 años.
- 👥 Análisis de mortalidad por género.
- 🫁 Pacientes que necesitaron asistencia respiratoria mecánica por edad.

---

## 📈 5. Visualización de Resultados

- 📍 **Línea temporal**: Casos diarios acumulados.  
- 🧱 **Histograma**: Intervalos clave entre eventos clínicos.  
- 🧮 **Gráfico de barras**: Asociación entre tipo de caso y desenlace.
- 🗺️ **Mapas geográficos**: Distribución de casos y severidad usando geopandas o folium.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# utilizamos parse_dates con la lista de fechas para que pandas las convierta a datetime
# y no tengamos que hacerlo manualmente después

df = pd.read_csv('./data/Covid19Casos.csv')


In [ ]:
df.dtypes

In [ ]:
# ya que vamos a trabajar unicamente con la provincia de Formosa
# vamos a delimitar dicho alcance
formosa_df = df[df['residencia_provincia_nombre'] == 'Formosa']
formosa_df = formosa_df.reset_index(drop=True)

df = formosa_df.copy()


In [ ]:
df.head()

In [ ]:
# Visualizamos la cantidad de registros y columnas del df

print(f'Cantidad de registros: {df.shape[0]}', end=' / ')
print(f'Cantidad de columnas: {df.shape[1]}')

# y los tipos de datos de cada columna

print('\nTipos de datos:')
print(df.dtypes)

In [ ]:
# hay muchas columnas que no nos interesan para este análisis,
# como por ej: el pais, y el nombre  de la provincia, 
# ya que hemos delimitado previamente el alcance
# tambien: origen_financiamiento, carga_provincia_id, ultima_actualizacion, residencia_departamento_id, y residencia_provincia_id

drop_cols = []


drop_cols.append('id_evento_caso')
drop_cols.append('residencia_pais_nombre')
drop_cols.append('residencia_provincia_nombre')
drop_cols.append('carga_provincia_nombre')
drop_cols.append('origen_financiamiento')
drop_cols.append('clasificacion')
drop_cols.append('carga_provincia_id')
drop_cols.append('ultima_actualizacion')
drop_cols.append('residencia_departamento_id')
drop_cols.append('residencia_provincia_id')

print(drop_cols)

In [ ]:
df.drop(columns=drop_cols, inplace=True)

print(f'Columnas restantes: {df.columns}')

In [ ]:
# Hemos eliminado las columnas que no nos interesan
# y ahora vamos a ver los tipos de datos nuevamente
# ya que todavia necesitamos convertir las fechas a datetime
df.dtypes

In [ ]:
print(f'Cantidad de registros: {df.shape[0]}', end=' / ')
print(f'Cantidad de columnas: {df.shape[1]}')

In [ ]:
renombres = {
    'residencia_departamento_nombre': 'departamento_residencia',
    'fecha_inicio_sintomas': 'fecha_inicio_sintomas',
    'sepi_apertura': 'sem_epidemiologica',
    'cuidado_intensivo': 'requirio_uci', 
    'fecha_cui_intensivo': 'fecha_ingreso_uci',
    'fallecido': 'indicador_fallecimiento',
    'fecha_fallecimiento': 'fecha_fallecimiento',
    'asistencia_respiratoria_mecanica': 'requirio_arm',
    'clasificacion_resumen': 'clasificacion_caso',
}

df.rename(columns=renombres, inplace=True)


In [ ]:
df.columns

In [ ]:
df['fecha_inicio_sintomas'] = pd.to_datetime(df['fecha_inicio_sintomas'], errors='coerce')
df['fecha_apertura'] = pd.to_datetime(df['fecha_apertura'], errors='coerce')
df['fecha_internacion'] = pd.to_datetime(df['fecha_internacion'], errors='coerce')
df['fecha_ingreso_uci'] = pd.to_datetime(df['fecha_ingreso_uci'], errors='coerce')
df['fecha_fallecimiento'] = pd.to_datetime(df['fecha_fallecimiento'], errors='coerce')
df['fecha_diagnostico'] = pd.to_datetime(df['fecha_diagnostico'], errors='coerce')


In [ ]:
# pudimos convertir las fechas a un tipo datetime
# y esto nos va a permitir trabajar mejor con los datos
# y hacer graficos de series de tiempo, por ejemplo
df.dtypes

In [ ]:

df['edad_años_meses'].value_counts()

In [ ]:
print(f"edad máxima: {df['edad'].max()}")
print(f"edad minima: {df['edad'].min()}")

In [ ]:
df[df['edad_años_meses'] == 'Meses' ].edad.unique()

In [ ]:
binarias = ['requirio_uci', 'requirio_arm', 'indicador_fallecimiento']
categoricas = ['sexo', 'departamento_residencia']


In [ ]:
for col in binarias:
    df[col] = df[col].apply(lambda x: 1 if x == 'SI' else 0)

In [ ]:
for col in categoricas:
    df[col] = df[col].astype('category')

In [ ]:
df['edad'] = pd.to_numeric(df['edad'], errors='coerce')

In [ ]:
df['edad_estandarizada'] = df.apply(
    lambda row: row['edad'] / 12 if row['edad_años_meses'] == 'Meses' else row['edad'],
    axis=1
)

In [ ]:
# Filtrar edades válidas
df = df[(df['edad_estandarizada'] >= 0) & (df['edad_estandarizada'] <= 105)]

In [ ]:
# Crear grupos etarios
df['grupo_edad'] = pd.cut(
    df['edad_estandarizada'],
    bins=[0, 2, 12, 18, 30, 50, 65, 100, 105],
    labels=['0-2', '3-12', '13-18', '19-30', '31-50', '51-65', '65-100', '100+']
)

In [ ]:
# infantes = df[df['edad_años_meses'] == 'Meses' ]
# df = df[df['edad_años_meses'] != 'Meses' ]

# Ya que los valores se en la misma columna, podemos eliminar la columna "edad_años_meses"
df.drop(columns='edad_años_meses', inplace=True)

In [ ]:
df['clasificacion_caso'] = df['clasificacion_caso'].astype('category')

In [ ]:
df.dtypes

In [ ]:
df.head(10)

In [ ]:
# exportamos el dataframe a un archivo csv
df.to_csv('./data/Covid19Casos_Formosa.csv', index=False)

In [ ]:
plt.figure(figsize=(10,6))
df['edad_estandarizada'].plot(kind='hist', bins=40, color='skyblue', edgecolor='black')
plt.title('Distribución de Edad (Formosa)')
plt.xlabel('Edad (años)')
plt.ylabel('Cantidad de casos')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
casos_departamento = df[df['clasificacion_caso'] == 'Confirmado'].groupby('departamento_residencia').size().sort_values()
plt.figure(figsize=(12,10))
casos_departamento.plot(kind='barh', color='steelblue', edgecolor='black')
plt.title('Casos Confirmados por Departamento - Formosa')
plt.xlabel('Cantidad de Casos Confirmados')
plt.ylabel('Departamento')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
df['edad_estandarizada'].plot(kind='hist', bins=20, color='slategray', edgecolor='black')
plt.title('Distribución de Edad (Agrupada) - Casos COVID-19 en Formosa')
plt.xlabel('Edad en años')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
df['grupo_edad'].value_counts().sort_index().plot(kind='bar', color='salmon', edgecolor='black')
plt.title('Casos por Grupo Etario (Formosa)')
plt.xlabel('Grupo de Edad')
plt.ylabel('Cantidad de Casos')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
casos_por_fecha = df[df['clasificacion_caso'] == 'Confirmado'].groupby('fecha_diagnostico').size()
plt.figure(figsize=(12,6))
casos_por_fecha.plot(kind='line', color='teal')
plt.title('Evolución Temporal de Casos Confirmados - Formosa')
plt.xlabel('Fecha de Diagnóstico')
plt.ylabel('Cantidad de Casos Confirmados')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
casos_por_semana = df.groupby('sem_epidemiologica').size()
plt.figure(figsize=(12,6))
casos_por_semana.plot(kind='bar', color='orange', edgecolor='black')
plt.title('Casos por Semana Epidemiológica - Formosa')
plt.xlabel('Semana Epidemiológica')
plt.ylabel('Cantidad de Casos')
plt.tight_layout()
plt.show()

In [ ]:
uci_por_edad = df[df['requirio_uci'] == True]['grupo_edad'].value_counts().sort_index()
plt.figure(figsize=(10,6))
uci_por_edad.plot(kind='bar', color='purple', edgecolor='black')
plt.title('Casos en UCI por Grupo Etario')
plt.xlabel('Grupo de Edad')
plt.ylabel('Cantidad de Casos en UCI')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
arm_por_edad = df[df['requirio_arm'] == True]['grupo_edad'].value_counts().sort_index()
plt.figure(figsize=(10,6))
arm_por_edad.plot(kind='area', color='darkred', edgecolor='black')
plt.title('Casos con Asistencia Respiratoria Mecánica por Grupo Etario')
plt.xlabel('Grupo de Edad')
plt.ylabel('Cantidad de Casos con ARM')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
fallecidos = df[df['indicador_fallecimiento'] == True]
plt.figure(figsize=(12,6))
fallecidos.groupby('grupo_edad', observed=False)['sexo'].value_counts().unstack().plot(kind='bar', stacked=True)
plt.title('Fallecimientos por Grupo Etario y Sexo')
plt.xlabel('Grupo de Edad')
plt.ylabel('Cantidad de Fallecidos')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()
